call TrainAndSave.py to train many models with different initialization of quadrupoles

In [26]:
import sys
sys.path.append("../../")

import subprocess
import time

import torch

call train script in parallel and receive losses

In [43]:
regexLoss = re.compile("final loss: (.*)")

def invokeTraining(parameters: list) -> list:
    procs = list()
    for param in parameters:
        # start training in seperate process
        run = subprocess.Popen(["python", "TrainAndSave.py", str(param[0]), str(param[1]), str(param[2])],
                               shell=False, stdin=None, stdout=subprocess.PIPE, stderr=subprocess.PIPE, close_fds=True)
        
        procs.append(run)
    
    print("processes started")
    
    # wait for results
    for proc in procs:
        while True:
            if proc.poll() is None:
                time.sleep(1)
                continue
            break
    
    # check return codes
    for proc in procs:
        if proc.returncode != 0:
            # look at stdout and stderr
            out, err = proc.communicate()
            
            print("stdout")
            print(out.decode("utf-8"))
            print("stderr")
            print(err.decode("utf-8"))
            
            # abort
            print("exit code: {}".format(proc.returncode))
            raise IOError("training process returned non-zero exit code")
    
    # get final losses
    outs = list()
    for proc in procs:
        out, err = proc.communicate()
        outs.append(out)
    
    losses = list()
    for out in outs:
        for line in out.decode("utf-8").splitlines():
            match = regexLoss.search(line)
            if match:
                print(match.group(1))
                loss = float(match.group(1))
                losses.append(loss)

In [44]:
parameters = [[4.3, 3.1, 1e-5], [4.4, 3.2, 1e-3],]

losses = invokeTraining(parameters)

processes started
9.198180174271036e-11
7.3404597645536605e+127


create initial fit data

In [50]:
10**(-4.5)

3.1622776601683795e-05